# 암환자 유전체 데이터 기반 암종 분류 AI 모델 개발


- '2024 생명연구자원 AI활용 경진대회'는 바이오 데이터를 기반으로 한 AI 기술의 문제 해결 능력을 탐구하는 것을 목표로 합니다. <br>이 대회는 바이오 분야에서 AI 활용의 저변을 확대하고, 복잡한 바이오 데이터를 효율적으로 분석 및 해석할 수 있는 AI 알고리즘 개발에 초점을 맞추고 있습니다. <br><br>
- 본 대회의 구체적인 과제는 암환자 유전체 데이터의 변이 정보를 활용하여 암종을 분류하는 AI 모델을 개발하는 것입니다. <br>참가자들은 제공된 학습 데이터셋(암환자 유전체 변이 정보)을 사용하여 특정 변이 정보를 바탕으로 암종을 정확하게 분류할 수 있는 AI 알고리즘을 개발해야 합니다. <br><br>
- 이 대회의 궁극적인 목적은 바이오 데이터의 활용도를 높이고, 바이오 분야에서 AI 기술의 적용 가능성을 극대화하며, 인공지능 기술이 실제 바이오 의료 문제 해결에 어떻게 기여할 수 있는지 탐구하는 것입니다.

# Import library

In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
import xgboost as xgb

# Load Data

In [4]:
train = pd.read_csv(r"C:\Users\USER\python\데이콘\train.csv")
test = pd.read_csv(r"C:\Users\USER\python\데이콘\test.csv")

# Data Preprocessing

In [5]:
# SUBCLASS 가 범주형이기 때문에 LabelEncoder 사용
le_subclass = LabelEncoder()
train['SUBCLASS'] = le_subclass.fit_transform(train['SUBCLASS'])

# 변환된 레이블 확인
for i, label in enumerate(le_subclass.classes_):
    print(f"원래 레이블: {label}, 변환된 숫자: {i}")

원래 레이블: ACC, 변환된 숫자: 0
원래 레이블: BLCA, 변환된 숫자: 1
원래 레이블: BRCA, 변환된 숫자: 2
원래 레이블: CESC, 변환된 숫자: 3
원래 레이블: COAD, 변환된 숫자: 4
원래 레이블: DLBC, 변환된 숫자: 5
원래 레이블: GBMLGG, 변환된 숫자: 6
원래 레이블: HNSC, 변환된 숫자: 7
원래 레이블: KIPAN, 변환된 숫자: 8
원래 레이블: KIRC, 변환된 숫자: 9
원래 레이블: LAML, 변환된 숫자: 10
원래 레이블: LGG, 변환된 숫자: 11
원래 레이블: LIHC, 변환된 숫자: 12
원래 레이블: LUAD, 변환된 숫자: 13
원래 레이블: LUSC, 변환된 숫자: 14
원래 레이블: OV, 변환된 숫자: 15
원래 레이블: PAAD, 변환된 숫자: 16
원래 레이블: PCPG, 변환된 숫자: 17
원래 레이블: PRAD, 변환된 숫자: 18
원래 레이블: SARC, 변환된 숫자: 19
원래 레이블: SKCM, 변환된 숫자: 20
원래 레이블: STES, 변환된 숫자: 21
원래 레이블: TGCT, 변환된 숫자: 22
원래 레이블: THCA, 변환된 숫자: 23
원래 레이블: THYM, 변환된 숫자: 24
원래 레이블: UCEC, 변환된 숫자: 25


In [6]:
import pandas as pd
import re

# 아미노산 코드와 그들의 성질을 매핑
amino_acid_properties = {
    'A': 'nonpolar',    # Alanine
    'R': 'positive',    # Arginine
    'N': 'polar',       # Asparagine
    'D': 'negative',    # Aspartic Acid
    'C': 'polar',       # Cysteine
    'Q': 'polar',       # Glutamine
    'E': 'negative',    # Glutamic Acid
    'G': 'nonpolar',    # Glycine
    'H': 'positive',    # Histidine
    'I': 'nonpolar',    # Isoleucine
    'L': 'nonpolar',    # Leucine
    'K': 'positive',    # Lysine
    'M': 'nonpolar',    # Methionine
    'F': 'aromatic',    # Phenylalanine
    'P': 'nonpolar',    # Proline
    'S': 'polar',       # Serine
    'T': 'polar',       # Threonine
    'W': 'aromatic',    # Tryptophan
    'Y': 'aromatic',    # Tyrosine
    'V': 'nonpolar',    # Valine
}

# 변이 유형을 분류하는 함수 정의
def classify_mutation(mutation):
    # 결측치 처리
    if pd.isnull(mutation):
        return None  # 또는 특정 코드로 지정 가능

    mutation = str(mutation).strip()

    # WT 체크
    if mutation == 'WT':
        return 0  # WT (Wild Type)

    # 프레임시프트 돌연변이 체크 ('fs' 포함)
    if 'fs' in mutation:
        return 5  # 프레임시프트 돌연변이

    # 중단 돌연변이 체크 ('*' 포함)
    if '*' in mutation:
        return 4  # 중단 돌연변이

    # 돌연변이 패턴 매칭 (예: 'R496Q', 'L1700L')
    match = re.match(r'^([A-Z])(\d+)([A-Z])$', mutation)
    if match:
        from_aa = match.group(1)  # 원래 아미노산
        position = match.group(2) # 위치 (사용하지 않음)
        to_aa = match.group(3)    # 변이된 아미노산

        # 침묵 돌연변이 체크 (아미노산이 동일한 경우)
        if from_aa == to_aa:
            return 1  # 침묵 돌연변이

        # 아미노산 성질 가져오기
        from_property = amino_acid_properties.get(from_aa)
        to_property = amino_acid_properties.get(to_aa)

        # 아미노산 코드가 유효한지 확인
        if from_property is None or to_property is None:
            return 6  # 알 수 없는 아미노산 코드

        # 보존적 돌연변이 체크 (아미노산 성질이 동일한 경우)
        if from_property == to_property:
            return 2  # 보존적 돌연변이
        else:
            return 3  # 비보존적 돌연변이
    else:
        # 패턴 매칭 실패한 경우
        return 6  # 매칭 실패한 경우 6 반환

# 다중 치환을 처리하는 함수 정의
def classify_multiple_mutations(mutation_string):
    # 결측치 처리
    if pd.isnull(mutation_string):
        return None  # 또는 특정 코드로 지정 가능

    # 변이 문자열을 공백으로 분리
    mutations = str(mutation_string).strip().split()

    labels = []
    for mutation in mutations:
        label = classify_mutation(mutation)
        if label is not None:
            labels.append(label)
    if labels:
        # 가장 높은 값을 반환
        return max(labels)
    else:
        return None  # 또는 특정 코드로 지정 가능


In [7]:
# 제외할 열 목록 (예시로 'ID'와 'SUBCLASS'를 제외)
exclude_cols = ['ID', 'SUBCLASS']

# 변이 데이터가 있는 열 목록
mutation_cols = [col for col in train.columns if col not in exclude_cols]

# 각 열에 함수 적용
for col in mutation_cols:
    train[col] = train[col].apply(classify_multiple_mutations)

# 결과 출력 (일부 열만 표시)
print(train.head())

           ID  SUBCLASS  A2M  AAAS  AADAT  AARS1  ABAT  ABCA1  ABCA2  ABCA3  \
0  TRAIN_0000         8    0     0      0      0     0      0      0      0   
1  TRAIN_0001        19    0     0      0      0     0      0      0      0   
2  TRAIN_0002        20    1     0      0      0     0      0      0      0   
3  TRAIN_0003         9    0     0      0      0     0      0      0      0   
4  TRAIN_0004         6    0     0      0      0     0      0      0      0   

   ...  ZNF292  ZNF365  ZNF639  ZNF707  ZNFX1  ZNRF4  ZPBP  ZW10  ZWINT  ZYX  
0  ...       0       0       0       0      0      0     0     0      0    0  
1  ...       0       0       0       0      0      0     0     0      0    0  
2  ...       0       0       0       0      0      0     0     0      0    0  
3  ...       0       0       0       0      0      0     0     0      0    0  
4  ...       0       0       0       0      0      0     0     0      0    0  

[5 rows x 4386 columns]


In [9]:
print(train.head(19))

            ID  SUBCLASS  A2M  AAAS  AADAT  AARS1  ABAT  ABCA1  ABCA2  ABCA3  \
0   TRAIN_0000         8    0     0      0      0     0      0      0      0   
1   TRAIN_0001        19    0     0      0      0     0      0      0      0   
2   TRAIN_0002        20    1     0      0      0     0      0      0      0   
3   TRAIN_0003         9    0     0      0      0     0      0      0      0   
4   TRAIN_0004         6    0     0      0      0     0      0      0      0   
5   TRAIN_0005        21    0     0      2      0     0      0      0      0   
6   TRAIN_0006         2    0     0      0      0     0      0      0      0   
7   TRAIN_0007        23    0     0      0      0     0      0      0      0   
8   TRAIN_0008        12    0     0      0      0     0      0      0      0   
9   TRAIN_0009        21    0     0      0      0     0      0      0      0   
10  TRAIN_0010        21    0     0      0      0     0      0      0      0   
11  TRAIN_0011         7    0     0     

In [10]:
# 제외할 열 목록 (예시로 'ID'와 'SUBCLASS'를 제외)
exclude_cols = ['ID', 'SUBCLASS']

# 변이 데이터가 있는 열 목록
mutation_cols = [col for col in train.columns if col not in exclude_cols]

# 각 열에 함수 적용
for col in mutation_cols:
    test[col] = test[col].apply(classify_multiple_mutations)

# 결과 출력 (일부 열만 표시)
print(test.head())

          ID  A2M  AAAS  AADAT  AARS1  ABAT  ABCA1  ABCA2  ABCA3  ABCA4  ...  \
0  TEST_0000    0     0      0      0     0      0      0      0      0  ...   
1  TEST_0001    0     0      0      0     0      3      0      0      0  ...   
2  TEST_0002    0     0      0      0     0      0      0      0      0  ...   
3  TEST_0003    0     0      0      0     0      0      0      0      0  ...   
4  TEST_0004    0     0      0      0     0      0      0      0      0  ...   

   ZNF292  ZNF365  ZNF639  ZNF707  ZNFX1  ZNRF4  ZPBP  ZW10  ZWINT  ZYX  
0       0       0       0       0      0      0     0     0      0    0  
1       0       0       0       0      0      5     0     0      0    0  
2       0       0       0       0      0      0     0     0      0    0  
3       0       0       0       0      0      0     0     0      0    0  
4       0       0       0       0      0      0     0     0      0    0  

[5 rows x 4385 columns]


# Model Define and Train

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb

# 1. 데이터 준비
# 타겟: 'SUBCLASS', 특징: 'SUBCLASS'와 'ID'를 제외한 나머지 열
X = train.drop(columns=['SUBCLASS', 'ID'])  # 특징 데이터 (SUBCLASS를 제외한 모든 열)
y = train['SUBCLASS']  # 타겟 데이터 (SUBCLASS)

# 2. 학습 세트와 테스트 세트로 데이터 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# 3. XGBoost 모델 생성
model = xgb.XGBClassifier(objective='multi:softmax', num_class=len(y.unique()), random_state=42, use_label_encoder=False)

# 4. 모델 학습
model.fit(X_train, y_train)

# 5. 예측
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# 6. 정확도 계산
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

# 7. 결과 출력
print(f"Train Accuracy: {train_accuracy * 100:.2f}%")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

c:\Users\USER\miniconda3\envs\machine\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:19:55] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Train Accuracy: 81.83%
Test Accuracy: 33.12%


# Inference

In [21]:
test = test.drop(columns=['ID'])
# X_encoded = test_X.copy()
# X_encoded[categorical_columns] = ordinal_encoder.transform(test_X[categorical_columns])

In [22]:
predictions = model.predict(test)

In [23]:
original_labels = le_subclass.inverse_transform(predictions)

# Submisson

In [24]:
submisson = pd.read_csv("./sample_submission.csv")

In [25]:
submisson["SUBCLASS"] = original_labels

In [26]:
submisson.to_csv('./baseline_submission.csv', encoding='UTF-8-sig', index=False)